In [57]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from Algorithmes.music import estimate_angles
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import itertools
from torchsummary import summary
from numpy.linalg import eigh
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary

In [2]:
nbSources = 1 # Nombre de sources  dans l'article ils fixent le nombre de sources à 2
nbSensors = 9 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels



#Pour l'échantillon d'entraînements ils utilisent une observation de chaque combinaison d'angles d'arrivée possible pour chaque SNR level, dans le cas d'une seule source, 2G+1 observations
#SNR ratio dans le papier est seulement low, de -20 à 0 DB avec un pas de 5
#Petite note : si on fait un modèle pour chaque SNR level, les performances augmentent légérements, mais cela ne vaut pas l'effort effectué
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = nbSensors
T = 100
correlation_List = [] # Liste des corrélations. Il y a une corrélation nécéssaire pour chaque paire distincte de sources différentes: 0 pour 1 source, 1 pour 2 sources, 3 pour 3 sources, 6 pour 4 sources etc...
# Ordre de remplisage de la correlation_List: de gauche à droite et ligne par ligne, moitié haut-droite de la matrice uniquement, puis symétrie de ces valeurs pour la moitié bas-gauche.

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split


#Paramètres pour la simulation

num_samples = 1000  # Nombre d'échantillons à générer #16290
L = nbSensors  # Nombre de capteurs
T = 100  # Nombre de points temporels
phi_max = 45  # φmax
rho = 1 # Résolution

# Grille des angles
G = np.arange(-phi_max, phi_max + rho, rho)


In [34]:
import numpy as np
from numpy.linalg import eigh

def generate_combinations(phi_max, rho, nb_sources):
    """Cette fonction sert à générer toutes les combinaisons d'angles possibles dans la plage de résolution"""
    # Générer une plage de valeurs possibles pour les signaux
    values = list(range(-int(phi_max / rho), int(phi_max / rho) + 1))
    
    # Générer toutes les combinaisons possibles de signaux
    all_combinations = list(itertools.product(values, repeat=nb_sources))
    
    # Supprimer les combinaisons où l'ordre ne compte pas
    unique_combinations = set()
    for combination in all_combinations:
        sorted_combination = tuple(sorted(combination))
        unique_combinations.add(sorted_combination)
    unique_combinations=list(unique_combinations)
        
    #supprimer les combinaisons ou deux signaux ont la même valeur
    combination_a_supp=[]
    for combination in unique_combinations:
        presents=[]
        for element in combination:
            if element not in presents:
                presents.append(element)
            else:
                combination_a_supp.append(combination)
    for combination in combination_a_supp:
        unique_combinations.remove(combination)
                
        
    
    return unique_combinations

def generate_music_spectrum(R, L, G, d=0.5, fs=1):
    """
    Calcule le spectre MUSIC à partir de la matrice de covariance R.

    :param R: Matrice de covariance estimée.
    :param L: Nombre de capteurs.
    :param G: Grille des angles (en degrés).
    :param d: Distance entre les éléments du réseau de capteurs (en longueurs d'onde).
    :param fs: Fréquence d'échantillonnage.
    :return: Spectre MUSIC pour les angles dans G.
    """
    # Calcul des valeurs propres et vecteurs propres
    eigvals, eigvecs = eigh(R)
    
    # Séparation des vecteurs du signal et du bruit
    E_noise = eigvecs[:, :L-2]  # Supposant 2 sources max
    
    # Calcul du spectre MUSIC
    P_music = np.zeros(len(G))
    for i, theta in enumerate(G):
        a = np.exp(np.arange(L) * 1j * 2 * np.pi * d * np.sin(np.radians(theta)) / fs)
        P_music[i] = 1 / np.abs(a.conj().T @ E_noise @ E_noise.conj().T @ a)
    
    # Normalisation du spectre
    P_music = P_music / np.max(P_music)
    
    return P_music

def generation_donnees( phi_max : int, rho : float, SNR : list, correlation : list, calibration ):
    angles_combination=generate_combinations(phi_max,rho,nbSources)
    num_samples=len(angles_combination)*len(SNR)
    
    # Initialisation des tableaux pour les données
    all_R_hat_with_phase = np.zeros((num_samples, 3, L, L))
    all_music_spectra = np.zeros((num_samples, len(G)))
    
    #On parcourt les différents SNR possibles
    i=0
    for snr in SNR:
        # Générer les angles d'arrivée et la matrice X comme décrit précédemment
        for angles in angles_combination:
            thetaList=list(angles)
            varList = [np.random.uniform(0, 1000000000000) for _ in range(nbSources)]
            X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, snr)# Votre fonction pour générer X, ajoutez les paramètres nécessaires
            
            # Calculer R_hat étendue avec phase
            R_hat_with_phase = generate_R_hat_with_phase(X)
            all_R_hat_with_phase[i] = R_hat_with_phase
            
            # Calculer les labels Z pour les angles générés
            R = all_R_hat_with_phase[i, :, :, 0] + 1j * all_R_hat_with_phase[i, :, :, 1]  # Reconstitution de la matrice complexe
            all_music_spectra[i] = music_method(X, nbSensors, nbSources)
            i+=1
    return all_R_hat_with_phase, all_music_spectra

# Initialisation des tableaux pour les données
all_R_hat_with_phase, all_music_spectra=generation_donnees(phi_max, rho, [15, 20, 25, 30],[],[])

SyntaxError: invalid syntax (1158272649.py, line 91)

In [35]:
import numpy as np
from numpy.linalg import eigh

def generate_combinations(phi_max, rho, nb_sources):
    """Cette fonction sert à générer toutes les combinaisons d'angles possibles dans la plage de résolution"""
    # Générer une plage de valeurs possibles pour les signaux
    values = list(range(-int(phi_max / rho), int(phi_max / rho) + 1))
    
    # Générer toutes les combinaisons possibles de signaux
    all_combinations = list(itertools.product(values, repeat=nb_sources))
    
    # Supprimer les combinaisons où l'ordre ne compte pas
    unique_combinations = set()
    for combination in all_combinations:
        sorted_combination = tuple(sorted(combination))
        unique_combinations.add(sorted_combination)
    unique_combinations=list(unique_combinations)
        
    #supprimer les combinaisons ou deux signaux ont la même valeur
    combination_a_supp=[]
    for combination in unique_combinations:
        presents=[]
        for element in combination:
            if element not in presents:
                presents.append(element)
            else:
                combination_a_supp.append(combination)
    for combination in combination_a_supp:
        unique_combinations.remove(combination)
                
        
    
    return unique_combinations

def generate_music_spectrum(R, L, G, d=0.5, fs=1):
    """
    Calcule le spectre MUSIC à partir de la matrice de covariance R.

    :param R: Matrice de covariance estimée.
    :param L: Nombre de capteurs.
    :param G: Grille des angles (en degrés).
    :param d: Distance entre les éléments du réseau de capteurs (en longueurs d'onde).
    :param fs: Fréquence d'échantillonnage.
    :return: Spectre MUSIC pour les angles dans G.
    """
    # Calcul des valeurs propres et vecteurs propres
    eigvals, eigvecs = eigh(R)
    
    # Séparation des vecteurs du signal et du bruit
    E_noise = eigvecs[:, :L-2]  # Supposant 2 sources max
    
    # Calcul du spectre MUSIC
    P_music = np.zeros(len(G))
    for i, theta in enumerate(G):
        a = np.exp(np.arange(L) * 1j * 2 * np.pi * d * np.sin(np.radians(theta)) / fs)
        P_music[i] = 1 / np.abs(a.conj().T @ E_noise @ E_noise.conj().T @ a)
    
    # Normalisation du spectre
    P_music = P_music / np.max(P_music)
    
    return P_music

def generation_donnees( phi_max : int, rho : float, SNR : list, correlation : list, calibration ):
    angles_combination=generate_combinations(phi_max,rho,nbSources)
    num_samples=len(angles_combination)*len(SNR)
    
    # Initialisation des tableaux pour les données
    all_R_hat_with_phase = np.zeros((num_samples, 3, L, L))
    all_music_spectra = np.zeros((num_samples, len(G)))
    
    #On parcourt les différents SNR possibles
    i=0
    for snr in SNR:
        # Générer les angles d'arrivée et la matrice X comme décrit précédemment
        for angles in angles_combination:
            thetaList=list(angles)
            varList = [np.random.uniform(0, 1000000000000) for _ in range(nbSources)]
            X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, snr)# Votre fonction pour générer X, ajoutez les paramètres nécessaires
            
            # Calculer R_hat étendue avec phase
            R_hat_with_phase = generate_R_hat_with_phase(X)
            all_R_hat_with_phase[i] = R_hat_with_phase
            
            # Calculer les labels Z pour les angles générés
            R = all_R_hat_with_phase[i, :, :, 0] + 1j * all_R_hat_with_phase[i, :, :, 1]  # Reconstitution de la matrice complexe
            all_music_spectra[i] = music_method(X, nbSensors, nbSources)
            i+=1
    return all_R_hat_with_phase, all_music_spectra

# Initialisation des tableaux pour les données
all_R_hat_with_phase, all_music_spectra=generation_donnees(phi_max, rho, [0,-5],[],[])

[[4.25616348e+11]]
[[9.4693619e+11]]
[[4.06655901e+11]]
[[9.16059299e+11]]
[[9.48813018e+11]]
[[3.2587842e+11]]
[[3.04540354e+11]]
[[9.31078352e+11]]
[[4.81744429e+11]]
[[7.43877855e+11]]
[[4.53716184e+11]]
[[2.58425699e+11]]
[[3.80674768e+11]]
[[4.84969271e+11]]
[[8.91864015e+10]]
[[4.59168303e+11]]
[[4.76718848e+11]]
[[5.40509174e+11]]
[[2.8818426e+11]]
[[9.75727696e+11]]
[[9.95806499e+11]]
[[2.51243387e+11]]
[[1.08715859e+11]]
[[3.82566098e+11]]
[[2.55551555e+11]]
[[7.63600303e+11]]
[[7.58568107e+11]]
[[7.43623382e+11]]
[[4.63773825e+11]]
[[5.90551974e+11]]
[[9.77957811e+11]]
[[9.09166727e+11]]
[[7.84227833e+11]]
[[5.90234121e+11]]
[[3.21676068e+11]]
[[7.13284828e+11]]
[[8.83647018e+11]]
[[5.96758842e+11]]
[[2.27839144e+10]]
[[3.84296909e+10]]
[[5.35441947e+11]]
[[2.41143823e+11]]
[[1.12495792e+11]]
[[7.54319203e+10]]
[[2.51829959e+11]]
[[1.4499185e+11]]
[[3.31019356e+11]]
[[9.20417958e+11]]
[[3.75630515e+11]]
[[1.07020687e+11]]
[[4.52060436e+11]]
[[9.84101351e+11]]
[[3.03721072e+11

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(all_R_hat_with_phase, all_music_spectra, test_size=0.1 , random_state=42)#test_size=0.1 selon papier
X_train, X_val, Y_train, Y_val= train_test_split(X_train, Y_train, test_size=0.1, random_state=1)

In [37]:
class DOADataset(Dataset):
    def __init__(self, covariance_matrices, spectra):
        """
        Initialisation du Dataset DOA.

        Args:
        - covariance_matrices (np.ndarray): Matrices de covariance étendues avec la phase.
          Forme attendue: (nb_samples, nbSensors, nbSensors, 3) pour les parties réelle, imaginaire, et la phase.
        - spectra (np.ndarray): Spectres MUSIC sous forme de vecteurs.
          Forme attendue: (nb_samples, len(G)), où len(G) est le nombre de points dans la grille DOA.
        """
        self.covariance_matrices = covariance_matrices
        self.spectra = spectra

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
        cov_matrix = self.covariance_matrices[idx]
        spectrum = self.spectra[idx]

        # Conversion en tenseur PyTorch
        cov_matrix = torch.tensor(cov_matrix, dtype=torch.float32)
        spectrum = torch.tensor(spectrum, dtype=torch.float32)

        return cov_matrix, spectrum

# Utilisation avec vos données X_train et Y_train
train_dataset = DOADataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Utilisation avec vos données X_val et Y_val pour l'ensemble de validation
val_dataset = DOADataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)  # Pas besoin de shuffle pour l'ensemble de validation

# Utilisation avec vos données X_test et Y_test pour l'ensemble de test
test_dataset = DOADataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [40]:


class DeepMusicModel(nn.Module):
    def __init__(self, num_classes=91):
        super(DeepMusicModel, self).__init__()

        # Couches convolutionnelles
        self.fc1 = nn.Linear(in_features=9*9*3, out_features=251)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5)
        self.conv8 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5)
        self.conv11 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5)

        # Couches d'activation ReLU
        self.relu2 = nn.ReLU()
        self.relu5 = nn.ReLU()
        self.relu8 = nn.ReLU()
        self.relu11 = nn.ReLU()

        # Couches de normalisation
        self.norm3 = nn.BatchNorm2d(256)
        self.norm6 = nn.BatchNorm2d(256)
        self.norm9 = nn.BatchNorm2d(256)
        self.norm12 = nn.BatchNorm2d(256)

        # Couche de convolution (kernel 1x1)
        self.conv15 = nn.Conv2d(256, 256, kernel_size=1)

        # Couche de dropout
        self.dropout16 = nn.Dropout(0.5)

        # Couches entièrement connectées (FC)
        self.fc17 = nn.Linear(in_features=7680, out_features=num_classes)
        

    def forward(self, x):
        x = self.fc1(x) 
        x = self.conv2(x)
        x = self.relu3(x)
        x = self.norm4(x)
        
        x = self.conv5(x)
        x = self.relu6(x)
        x = self.norm7(x)
        
        x = self.conv8(x)
        x = self.relu9(x)
        x = self.norm10(x)
        
        x = self.conv11(x)
        x = self.relu12(x)
        x = self.norm13(x)
        
        x = self.conv14(x)
                
        x = self.dropout16(x)
        x = x.view(x.size(0), -1)  # Aplatir pour FC
        x = self.fc17(x)
        

        return x


# Création de l'instance du modèle
model = DeepMusicModel()

# Définition de l'optimiseur et de la fonction de perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DeepMusicModel(nn.Module):
    def __init__(self):
        super(DeepMusicModel, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(num_features=256)
        
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(num_features=256)
        
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256)
        
        # Define the fourth convolutional layer
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=256)
        
        # Assuming the spatial dimensions (height and width) are reduced to 1x1 after the convolutions
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=20736, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=91)  # Example output size
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
        # Softmax is typically applied in the last layer if this is for classification
        # For regression or other tasks, it may not be necessary
        # self.softmax = nn.Softmax(dim=1)  # Uncomment if needed for classification

    def forward(self, x):
        # Apply the first convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn1(self.conv1(x)))
        
        # Apply the second convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn2(self.conv2(x)))
        
        # Apply the third convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn3(self.conv3(x)))
        
        # Apply the fourth convolutional layer and normalization, followed by ReLU
        x = F.relu(self.bn4(self.conv4(x)))
        
        # Flatten the tensor for the fully connected layer
        x = torch.flatten(x, 1)
        
        # Apply the first fully connected layer
        x = F.relu(self.fc1(x))
        
        # Apply the dropout layer
        x = self.dropout(x)
        
        # Apply the second fully connected layer
        x = self.fc2(x)
        
        # Apply softmax if this is for classification
        # x = self.softmax(x)  # Uncomment if needed for classification
        
        return x

# Create the CNN model
model = DeepMusicModel()

# Définition de l'optimiseur et de la fonction de perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [67]:
def initialize_cnns(num_subregions=10, input_size=91):
    cnns = [DeepMusicModel() for _ in range(num_subregions)]
    return cnns

# Example training loop for one CNN (extend this for training multiple CNNs)
def train_cnn(model, train_loader, epochs=100):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    model.train()
    for epoch in range(epochs):
        for data, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

#Boucle d'entraînement

def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)

    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    for data, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')


#Boucle de tests


def test_loop(dataloader, model, loss_fn, device):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct, MSE = 0, 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            _, angles_predit = pred.topk(nbSources, dim=1)
            _,angles_correct=top_values, top_indices = y.topk(nbSources, dim=1)
            correct += (angles_predit==angles_correct).type(torch.float).sum().item()
            #Calcul de la somme des carrés des différences entre prédictions et réalité pour la RMSE
            for angle in range(nbSources):
                MSE+=(angles_correct[angle]-angles_predit[angle])**2

    test_loss /= num_batches
    correct /= size
    RMSE=math.sqrt(MSE/(len(test_dataloader)*nbSources))
    print(f"\nTest set: \n  Accuracy: {(100 * correct): >0.1f}%, Avg loss: {test_loss: >8f} \n RMSE : {RMSE: >8f}")
    return test_loss, correct

def train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs, device):
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loop(train_dataloader, model, loss_fn, optimizer, device)
        test_loop(test_dataloader, model, loss_fn, device)
        scheduler.step()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 35)

In [61]:
#Définition du device

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cpu device


In [62]:
#Définition des hyperparamètres, choisis en fonction de l'article DOA estimator

learning_rate = 0.001
num_epochs = 10 # à monter à 200 pour le training final selon l'article
loss_fn = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [63]:
num_subregions = 8
cnns = initialize_cnns(num_subregions)

In [65]:
i = 1
for cnn in cnns:
     print("Model {}".format(i))
     train(cnn, train_loader, test_loader, loss_fn, optimizer, num_epochs, device)
     i +=1

Model 1
Epoch 1
-------------------------------


RuntimeError: all elements of target should be between 0 and 1

In [25]:
print(X_train.shape, Y_train.shape)

(146, 3, 9, 9) (146, 91)
